In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [10]:
pseudo = pd.read_csv("../input/30dml-pseudo2/xgb_submission_k.txt")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
              'objective': 'reg:squarederror',
               'learning_rate':trial.suggest_uniform('learning_rate', 3.86e-2, 8.86e-2),
               'n_estimators':N_ESTIMATORS,
               'subsample': trial.suggest_uniform('subsample', 0.832, 1),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.001, 0.167),
              'max_depth':trial.suggest_int('max_depth', 5, 9),
              'lambda':trial.suggest_uniform('lambda', 5.4, 13.4),
              'alpha':trial.suggest_uniform('alpha', 53.34, 65.34),
           'min_child_weight':trial.suggest_uniform('min_child_weight', 313, 423),
           'importance_type': 'total_gain',
            'tree_method': 'hist'
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  xgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = xgb.XGBRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        
    )


    xgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, xgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-24 06:32:10,600] A new study created in memory with name: no-name-c70ea56a-6085-47fa-9496-748cce9ea4aa


[0]	validation_0-rmse:7.34123
[1000]	validation_0-rmse:0.71913
[2000]	validation_0-rmse:0.71368
[3000]	validation_0-rmse:0.71186
[4000]	validation_0-rmse:0.71107
[5000]	validation_0-rmse:0.71071
[6000]	validation_0-rmse:0.71054
[7000]	validation_0-rmse:0.71041
[8000]	validation_0-rmse:0.71036
[9000]	validation_0-rmse:0.71032
[10000]	validation_0-rmse:0.71028
[11000]	validation_0-rmse:0.71027
[11171]	validation_0-rmse:0.71027
[0]	validation_0-rmse:7.34117
[1000]	validation_0-rmse:0.72095
[2000]	validation_0-rmse:0.71428
[3000]	validation_0-rmse:0.71219
[4000]	validation_0-rmse:0.71132
[5000]	validation_0-rmse:0.71087
[6000]	validation_0-rmse:0.71067
[7000]	validation_0-rmse:0.71051
[8000]	validation_0-rmse:0.71044
[9000]	validation_0-rmse:0.71037
[10000]	validation_0-rmse:0.71033
[11000]	validation_0-rmse:0.71030
[12000]	validation_0-rmse:0.71028
[13000]	validation_0-rmse:0.71026
[14000]	validation_0-rmse:0.71025
[14358]	validation_0-rmse:0.71025


[I 2021-08-24 06:52:43,532] Trial 0 finished with value: 0.7102556606197346 and parameters: {'learning_rate': 0.05643669245393019, 'subsample': 0.909640970334601, 'colsample_bytree': 0.01617940084638739, 'max_depth': 9, 'lambda': 11.860785180590966, 'alpha': 56.52662496144876, 'min_child_weight': 347.49662757408333}. Best is trial 0 with value: 0.7102556606197346.


[0]	validation_0-rmse:7.13210
[1000]	validation_0-rmse:0.71062
[2000]	validation_0-rmse:0.71008
[2691]	validation_0-rmse:0.71000
[0]	validation_0-rmse:7.13208
[1000]	validation_0-rmse:0.71063
[2000]	validation_0-rmse:0.71008
[2922]	validation_0-rmse:0.71001


[I 2021-08-24 06:58:58,791] Trial 1 finished with value: 0.7099389093558335 and parameters: {'learning_rate': 0.0835906348362928, 'subsample': 0.968184012042584, 'colsample_bytree': 0.0742320074827586, 'max_depth': 7, 'lambda': 8.274169082560896, 'alpha': 60.94051839189858, 'min_child_weight': 330.1258476888272}. Best is trial 1 with value: 0.7099389093558335.


...

[0]	validation_0-rmse:7.36511
[1000]	validation_0-rmse:0.71155
[2000]	validation_0-rmse:0.71027
[3000]	validation_0-rmse:0.70996
[4000]	validation_0-rmse:0.70985
[5000]	validation_0-rmse:0.70980
[5033]	validation_0-rmse:0.70980
[0]	validation_0-rmse:7.36511
[1000]	validation_0-rmse:0.71189
[2000]	validation_0-rmse:0.71034
[3000]	validation_0-rmse:0.71000
[4000]	validation_0-rmse:0.70987
[4913]	validation_0-rmse:0.70984


[I 2021-08-24 12:21:31,281] Trial 28 finished with value: 0.7097759154492443 and parameters: {'learning_rate': 0.05333328571035907, 'subsample': 0.9564314639533431, 'colsample_bytree': 0.062443135348732914, 'max_depth': 7, 'lambda': 8.7726421045735, 'alpha': 55.04787383414727, 'min_child_weight': 357.04103309671245}. Best is trial 21 with value: 0.7097666048475235.


[0]	validation_0-rmse:7.32661
[1000]	validation_0-rmse:0.71919
[2000]	validation_0-rmse:0.71379
[3000]	validation_0-rmse:0.71194
[4000]	validation_0-rmse:0.71113
[5000]	validation_0-rmse:0.71076
[6000]	validation_0-rmse:0.71059
[7000]	validation_0-rmse:0.71045
[8000]	validation_0-rmse:0.71040
[9000]	validation_0-rmse:0.71035
[10000]	validation_0-rmse:0.71031
[11000]	validation_0-rmse:0.71029
[11775]	validation_0-rmse:0.71029
[0]	validation_0-rmse:7.32660
[1000]	validation_0-rmse:0.72098
[2000]	validation_0-rmse:0.71431
[3000]	validation_0-rmse:0.71223
[4000]	validation_0-rmse:0.71139
[5000]	validation_0-rmse:0.71093
[6000]	validation_0-rmse:0.71070
[7000]	validation_0-rmse:0.71053
[8000]	validation_0-rmse:0.71045
[9000]	validation_0-rmse:0.71039
[10000]	validation_0-rmse:0.71034
[11000]	validation_0-rmse:0.71032
[12000]	validation_0-rmse:0.71029
[12463]	validation_0-rmse:0.71029


[I 2021-08-24 12:40:53,167] Trial 29 finished with value: 0.7102851173275533 and parameters: {'learning_rate': 0.05833220549974259, 'subsample': 0.958385095249468, 'colsample_bytree': 0.010480640968136001, 'max_depth': 7, 'lambda': 8.937367797856679, 'alpha': 56.918900585298424, 'min_child_weight': 367.745304955875}. Best is trial 21 with value: 0.7097666048475235.


Number of finished trials: 30
Best trial: {'learning_rate': 0.05889646321820302, 'subsample': 0.9696632151770538, 'colsample_bytree': 0.06416088812643116, 'max_depth': 7, 'lambda': 8.937879163296444, 'alpha': 54.38293217857195, 'min_child_weight': 324.6738529059135}


In [14]:
study.best_params

{'learning_rate': 0.05889646321820302,
 'subsample': 0.9696632151770538,
 'colsample_bytree': 0.06416088812643116,
 'max_depth': 7,
 'lambda': 8.937879163296444,
 'alpha': 54.38293217857195,
 'min_child_weight': 324.6738529059135}

# Log

====== Ordinal encoding =========

0.710263730055926 no noise ver2

0.7098699076215974 no noise ver5 (narrow space)

0.7097806935091454 no noise ver6 (narrow space)

0.7128654211262124 row-wise noise ver3

0.7131624379749847 random noise ver4

====== One-hot encoding =========

0.7101457824816443 no noise ver7

0.709767447519238 no noise ver10 (narrow space) final

0.7131014270509544 random noise ver8

0.7132486107908714 row-wise noise ver9


==========================================
=========== Changed pseudo ===============

====== One-hot encoding =========

0.7102184927322749 no noise ver13

0.7098602061085649 no noise ver15 (narrow space)